In [1]:
import pandas as pd
from ete3 import NCBITaxa
from ete3 import Tree

In [2]:
# Open taxonomic classification given to me by Fracois
df = pd.read_csv("Scion16_tax_Primer.csv")

In [4]:
df

,Unnamed: 0,Genus,Family,Order,Class,Phylum,Kingdom
0,ASV100,Hyphomonadaceae,Hyphomonadaceae,Caulobacterales,Alphaproteobacteria,Proteobacteria,Bacteria
1,ASV1000,Rhodobacteraceae,Rhodobacteraceae,Rhodobacterales,Alphaproteobacteria,Proteobacteria,Bacteria
2,ASV10000,Halioglobus,Halieaceae,Cellvibrionales,Gammaproteobacteria,Proteobacteria,Bacteria
3,ASV100000,OM190,OM190,OM190,OM190,Planctomycetes,Bacteria
4,ASV100001,Candidatus_Kaiserbacteria,Candidatus_Kaiserbacteria,Candidatus_Kaiserbacteria,Parcubacteria,Patescibacteria,Bacteria
...,...,...,...,...,...,...,...
123773,ASV99994,WCHB1-41,WCHB1-41,WCHB1-41,Kiritimatiellae,Kiritimatiellaeota,Bacteria
123774,ASV99996,OM190,OM190,OM190,OM190,Planctomycetes,Bacteria
123775,ASV99997,OM190,OM190,OM190,OM190,Planctomycetes,Bacteria
123776,ASV99998,Cellvibrionales,Cellvibrionales,Cellvibrionales,Gammaproteobacteria,Proteobacteria,Bacteria


In [5]:
# Get a list with all unique genera
family_list = df['Family'].unique()

In [6]:
# Check the number of unique genera
len(family_list)

728

In [7]:
# Initialize ETE3
ncbi = NCBITaxa()

In [8]:
# Convert genus name to NCBI tax id
tax_id_list = []
for family in family_list:
    name2taxid = ncbi.get_name_translator([str(family)])
    name2taxid = list(name2taxid.values())
    if len(name2taxid) > 0:
        tax_id_list.append(name2taxid[0][0])
len(tax_id_list)

357

In [9]:
# Build the tree
t = ncbi.get_topology(tax_id_list, intermediate_nodes=True)
t.write(format=8, outfile="tree_family.nw")

In [10]:
# Open results file from PlasticDB and create a dictionary with all genera names that had matches in the PlasticDB database
with open("j4369676_genus.tsv") as f:
    f = f.readlines()
    genus_dict = {}
    for line in f[1:]:
        if len(line.split("\t")) > 1:
            genus = line.split("\t")[1]
            if genus not in genus_dict:
                genus_dict[genus] = ""

In [11]:
# Convert genus name to NCBI tax id 
for genus, value in genus_dict.items():
    name2taxid = ncbi.get_name_translator([str(genus)])
    name2taxid = list(name2taxid.values())
    if len(name2taxid) > 0:
        genus_dict[genus] =  name2taxid[0][0]

In [22]:
# Get family tax ids for all genera
family_dict = {}
for item, value in genus_dict.items():
    for taxid in ncbi.get_lineage(int(value)):
        if ncbi.get_rank([taxid])[taxid] == "family":
            if taxid not in family_dict:
                family_dict[taxid] = taxid
                print(taxid)

641
213483
267891
267890
49546
267888
186817
72275
468
135621
28256
119060
194924
224372
75682
1762
1300
186822
85025
186824
135620
815
80864
1843490
31979
28056
41297
1268
32033
84566
31989
90964
85023
76892
2062
213421
506
186806
2070
41295
1903411
543


In [23]:
# Create annotation file to add stars to plastic degraders
labels_txt = open("degraders_symbols_family.txt", "w")
labels_txt.write("""DATASET_SYMBOL
SEPARATOR COMMA
DATASET_LABEL,All plastics
COLOR,#00ff00
MAXIMUM_SIZE,5
DATA
#100379,3,5,#0000ff,0,0
""")

for family, tax_id in family_dict.items():
            labels_txt.write(str(tax_id)+",3,50,#ff0000,1,1\n")
        
labels_txt.close()

In [24]:
# Create another genus dict with only the genera that were reported to degrade PBS
with open("j4369676_genus.tsv") as f:
    f = f.readlines()
    genus_dict = {}
    for line in f[1:]:
        if len(line.split("\t")) > 1:
            genus = line.split("\t")[1]
            plastic = line.split("\t")[3]
            if genus not in genus_dict and plastic == "PBS":
                genus_dict[genus] = ""

In [25]:
# Convert genus name to NCBI tax id 
for genus, value in genus_dict.items():
    name2taxid = ncbi.get_name_translator([str(genus)])
    name2taxid = list(name2taxid.values())
    if len(name2taxid) > 0:
        genus_dict[genus] =  name2taxid[0][0]

In [26]:
# Get family tax ids for all genera
family_dict = {}
for item, value in genus_dict.items():
    for taxid in ncbi.get_lineage(int(value)):
        if ncbi.get_rank([taxid])[taxid] == "family":
            if taxid not in family_dict:
                family_dict[taxid] = taxid
                print(taxid)

186817
135621
186822
80864
2062
186824


In [27]:
# Create annotation file to add stars to plastic degraders
labels_txt = open("PBS_symbols_family.txt", "w")
labels_txt.write("""DATASET_SYMBOL
SEPARATOR COMMA
DATASET_LABEL,All plastics
COLOR,#00ff00
MAXIMUM_SIZE,5
DATA
#100379,3,5,#0000ff,0,0
""")

for family, tax_id in family_dict.items():
            labels_txt.write(str(tax_id)+",3,50,#0000ff,1,1\n")
        
labels_txt.close()